<a href="https://colab.research.google.com/github/jscienciadados/ciencia-dados/blob/main/BOW_PNL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color="SlateBlue">Processamento de Linguagem Natural</font>

# Modelo de Classificação de Idiomas de Sentenças com Bag of Words e Pytorch

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch import nn, optim


# Preparando os Dados

In [2]:
# Dados de treino
dados_treino = [("Tenho vinte paginas de leitura".lower().split(), "Portuguese"),
                ("I will visit the library".lower().split(), "English"),
                ("I am reading a book".lower().split(), "English"),
                ("This is my favourite chapter".lower().split(), "English"),
                ("Estou na biblioteca lendo meu livro preferido".lower().split(), "Portuguese"),
                ("Gosto de livros sobre viagens".lower().split(), "Portuguese")]


In [3]:
dados_treino

[(['tenho', 'vinte', 'paginas', 'de', 'leitura'], 'Portuguese'),
 (['i', 'will', 'visit', 'the', 'library'], 'English'),
 (['i', 'am', 'reading', 'a', 'book'], 'English'),
 (['this', 'is', 'my', 'favourite', 'chapter'], 'English'),
 (['estou', 'na', 'biblioteca', 'lendo', 'meu', 'livro', 'preferido'],
  'Portuguese'),
 (['gosto', 'de', 'livros', 'sobre', 'viagens'], 'Portuguese')]

In [4]:
# Dados de teste
dados_teste = [("Estou lendo".lower().split(), "Portuguese"),
               ("This is not my favourite book".lower().split(), "English")]


In [5]:
# Prepara o dicionário do vocabulário

# Dicionário para o vocabulário
dict_vocab = {}

# Contadoor
i = 0

# Loop pelos dados de treino e teste
for palavras, idiomas in dados_treino + dados_teste:
    for palavra in palavras:
        if palavra not in dict_vocab:
            dict_vocab[palavra] = i
            i += 1


In [6]:
# Visualiza p vocabulario
print(dict_vocab)

{'tenho': 0, 'vinte': 1, 'paginas': 2, 'de': 3, 'leitura': 4, 'i': 5, 'will': 6, 'visit': 7, 'the': 8, 'library': 9, 'am': 10, 'reading': 11, 'a': 12, 'book': 13, 'this': 14, 'is': 15, 'my': 16, 'favourite': 17, 'chapter': 18, 'estou': 19, 'na': 20, 'biblioteca': 21, 'lendo': 22, 'meu': 23, 'livro': 24, 'preferido': 25, 'gosto': 26, 'livros': 27, 'sobre': 28, 'viagens': 29, 'not': 30}


In [7]:
# Tamanho do corpus
tamanho_corpus = len(dict_vocab)

In [8]:
tamanho_corpus

31

In [18]:
# numero de idioma
idiomas = 2
# indice para os idiomas
label_index = {"Portuguese": 0, "English": 1}

#<font color="Goldenrod">Construção do Modelo</font>

In [11]:
# Classe para o modelo BOW de classificação
class ModeloBOW(nn.Module):  

    # Método construtor
    def __init__(self, lista_idiomas, tamanho_do_corpus):
        super(ModeloBOW, self).__init__()
        self.linear = nn.Linear(tamanho_do_corpus, lista_idiomas)

    # Feed Forward
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim = 1)


In [12]:
# Função para criar o vetor BOW necessário para o treinamento
def cria_bow_vetor(sentence, word_index):
    word_vec = torch.zeros(tamanho_corpus)
    for word in sentence:
        word_vec[dict_vocab[word]] += 1
    return word_vec.view(1, -1)


In [13]:
# Função para criar a variável target
def cria_target(label, label_index):
    return torch.LongTensor([label_index[label]])


In [14]:
# Cria o modelo
modelo = ModeloBOW(idiomas, tamanho_corpus)


In [15]:
# Função de perda (loss)
loss_function = nn.NLLLoss()

In [16]:
# Otimizador
optimizer = optim.SGD(modelo.parameters(), lr = 0.1)


#<font color="Crimson">Treinamento do Modelo</font>

In [19]:
# Loop de treinamentoo
for epoch in range(100):
   
    for sentence, label in dados_treino:

        modelo.zero_grad()

        bow_vec = cria_bow_vetor(sentence, dict_vocab)
        target = cria_target(label, label_index)

        log_probs = modelo(bow_vec)

        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
       
    if epoch % 10 == 0:
        print('Epoch: ', str(epoch+1),', Loss: ' + str(loss.item()))


Epoch:  1 , Loss: 0.5234674215316772
Epoch:  11 , Loss: 0.07969143986701965
Epoch:  21 , Loss: 0.04177490994334221
Epoch:  31 , Loss: 0.02820836752653122
Epoch:  41 , Loss: 0.02127061039209366
Epoch:  51 , Loss: 0.017064061015844345
Epoch:  61 , Loss: 0.014243250712752342
Epoch:  71 , Loss: 0.012220979668200016
Epoch:  81 , Loss: 0.010700704529881477
Epoch:  91 , Loss: 0.009516234509646893


#<font color="OrangeRed">Previsões e Avaliação do Modelo</font>

In [20]:
# Função para previsões
def faz_previsao(data):

    with torch.no_grad():
        sentence = data[0]
        label = data[1]
        bow_vec = cria_bow_vetor(sentence, dict_vocab)
        log_probs = modelo(bow_vec)
        print(sentence)
        print('Probabilidade de ser o label: ' + label, 'é igual a: ',  np.exp(log_probs))


In [21]:
# Previsão com a primeira sentença
faz_previsao(dados_teste[0])

['estou', 'lendo']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.7946, 0.2054]])


In [22]:
dados_teste[0]

(['estou', 'lendo'], 'Portuguese')

In [23]:
# previsaão com a segunda setença
faz_previsao(dados_teste[1])

['this', 'is', 'not', 'my', 'favourite', 'book']
Probabilidade de ser o label: English é igual a:  tensor([[0.0103, 0.9897]])


#<font color="Brown">Previsões com Novas Frases</font>

In [32]:
# Nova frase
novas_frases = [("Tenho livros sobre viagens".lower().split(), "Portuguese"),
                ("Estou escrevendo".lower().split(), "Portuguese"),
                ("Gosto de biblioteca".lower().split(), "Portuguese"),
                ("The Bible  book that inspires man the most".lower().split(), "English")]


In [33]:
novas_frases

[(['tenho', 'livros', 'sobre', 'viagens'], 'Portuguese'),
 (['estou', 'escrevendo'], 'Portuguese'),
 (['gosto', 'de', 'biblioteca'], 'Portuguese'),
 (['the', 'bible', 'book', 'that', 'inspires', 'man', 'the', 'most'],
  'English')]

In [28]:
faz_previsao(novas_frases[0])

['tenho', 'livros', 'sobre', 'viagens']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.9516, 0.0484]])


In [31]:
faz_previsao(novas_frases[2])


['gosto', 'de', 'biblioteca']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.9690, 0.0310]])
